In [ ]:
import numpy as np
import pandas as pd
import torch
import torch.nn as nn
import torch.optim as optim
from scipy.integrate import quad
from scipy.integrate import solve_ivp
from scipy.optimize import root_scalar
import matplotlib.pyplot as plt
from scipy.special import expit

In [ ]:
# Parameters
lambd = 3.36251543
rho = 0.19183673
u0 = 0.12950433
gamma1 = 1.0

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [ ]:
# Define F(u)
def F(u):
    return 0.5 * u**2 - (1/3) * u**3

# Calculate rho from q using the given relationship
def compute_q(rho, gamma):
    def equation_to_solve(q):
        if q <= 0 or q >= rho:
            return 1e6  # prevent invalid region
        return 0.5 * (2 * F(q) + gamma**2 * q**2) - F(rho)

    result = root_scalar(equation_to_solve, bracket=[1e-4, rho - 1e-4], method='brentq')
    return result.root if result.converged else np.nan

# Given q and rho, find lambda such that the integral equals 0.5
def find_lambda_for_q_rho(q, rho):
    def objective(lambd):
        if lambd <= 0:
            return 1e6
        try:
            integrand = lambda u: 1.0 / np.sqrt(2 * lambd * (F(rho) - F(u)))
            val, _ = quad(integrand, q, rho - 1e-8, limit=200)
            return val - 0.5
        except Exception:
            return 1e6

    result = root_scalar(objective, bracket=[1e-3, 1000], method='brentq')
    return result.root if result.converged else None

# Define the ODE
def ode1(t, u, rho, lambd):
    diff = F(rho) - F(u)
    return np.sqrt(2 * lambd * diff) if diff > 0 else 0.0

# Set gamma value
gamma = 1.0  # Adjust as needed

# Choose a range of q values and compute rho, lambda
rho_values = np.linspace(0.1, 1.0, 50)
q_values = []
lambda_values = []

for rho in rho_values:
    try:
        q = compute_q(rho, gamma)
        lambd = find_lambda_for_q_rho(q, rho)
        q_values.append(q)
        lambda_values.append(lambd)
    except Exception:
        q_values.append(np.nan)
        lambda_values.append(np.nan)
# rho vs q plot
plt.figure(figsize=(10, 6))
plt.plot(rho_values, q_values, label=fr"$\rho(q)$ for $\gamma$ = {gamma}", color='green')
plt.ylabel("q")
plt.xlabel("ρ")
plt.title("q vs ρ")
plt.grid(True)
plt.legend()
plt.tight_layout()
plt.show()

# Print some q rho pairs
print(f"{'q':>8} {'rho':>12}")
print("-" * 22)
for i in range(0, len(rho_values), max(1, len(rho_values)//10)):
    rho = rho_values[i]
    q = q_values[i]
    if not np.isnan(rho):
        print(f"{q:8.8f} {rho:12.8f}")

# Plot lambda(ρ)
plt.figure(figsize=(10, 6))
plt.plot(lambda_values, rho_values, label=r'$\lambda(q)$ for $\gamma$ = {}'.format(gamma))
plt.ylabel("ρ u(max)")
plt.xlabel("λ")
plt.xlim(0,50)
plt.grid(True)
plt.legend()
plt.title("λ vs ρ from quadrature method")
plt.tight_layout()
plt.show()

# Print some lambda values for q rho pairs
print(f"{'q':>8} {'rho':>12} {'lambda':>12}")
print("-" * 36)
for i in range(0, len(rho_values), max(1, len(rho_values)//10)):
    q = q_values[i]
    rho = rho_values[i]
    lambd = lambda_values[i]
    if not (np.isnan(rho) or np.isnan(lambd)):
        print(f"{q:8.8f} {rho:12.8f} {lambd:12.8f}")

# Plot a few u(t) curves
selected_lambdas = [3.36, 5.85, 9.53, 19.38, 35.26]
plt.figure(figsize=(10, 6))
for lam in selected_lambdas:
    # Find closest lambda in computed lambda_values
    arr = np.array(lambda_values)
    idx = np.nanargmin(np.abs(arr - lam))

    q = q_values[idx]
    rho = rho_values[idx]
    lambd = lambda_values[idx]

    if not np.isnan(lambd):
        sol = solve_ivp(ode1, [0, 0.5], [q], args=(rho, lambd),
                        dense_output=True, rtol=1e-9,
                        atol=1e-12)
        t_half = np.linspace(0, 0.5, 100)
        u_half = sol.sol(t_half)[0]
        t_full = np.concatenate([t_half, 1 - t_half[::-1]])
        u_full = np.concatenate([u_half, u_half[::-1]])

        plt.plot(t_full, u_full, label=f"λ={lam:.2f}", alpha=0.8)

plt.xlabel("t")
plt.ylabel("u(t)")
plt.ylim(0,1)
plt.grid(True)
plt.legend()
plt.title(f"Quadrature Solutions u(t) for selected λ values, γ = {gamma}")
plt.tight_layout()
plt.show()

In [ ]:
lambda_val = [2.88349928, 3.36251543,
              3.97827418, 4.77695263,
              5.84805224, 7.34032295,
              9.53230192, 13.01886624,
              19.37997763, 35.26398748]

# Transcendental function to solve for mu
def f(mu, lambda_val, gamma1):
    term1 = (lambda_val * gamma1**2 - mu**2) * np.sin(mu)
    term2 = 2 * np.sqrt(lambda_val) * gamma1 * mu * np.cos(mu)
    return term1 + term2

# Find root mu for the principal eigenvalue
def find_mu(lambda_val, gamma1):
    mus = np.linspace(0.1, 20, 1000)
    for i in range(len(mus) - 1):
        a, b = mus[i], mus[i + 1]
        if f(a, lambda_val, gamma1) * f(b, lambda_val, gamma1) < 0:
            result = root_scalar(f, args=(lambda_val, gamma1), bracket=(a, b), method='brentq')
            if result.converged:
                return result.root
    return None

fig, axs = plt.subplots(len(lambda_val), 1, figsize=(8, 2*len(lambda_val)))

for idx, lam in enumerate(lambda_val):
    mu = find_mu(lam, gamma1)
    if mu is None:
        print(f"Could not find a valid root for mu for λ={lam}")
        continue
    C = np.sqrt(lam) * gamma1 / mu
    phi = lambda x: np.cos(mu * x) + C * np.sin(mu * x)
    x_vals = np.linspace(0, 1, 1000)
    phi_vals = phi(x_vals)
    phi_vals_normalized = phi_vals / np.max(np.abs(phi_vals))
    phi_min = np.min(phi_vals_normalized)

    m1 = (-(mu**2 - lam) / lam) * 0.5
    m2 = (-(mu**2 - lam) / (lam * phi_min))

    sub_phi = phi_vals_normalized * m1
    super_phi = phi_vals_normalized * m2

    print(f"λ ≈ {lam}, σ ≈ {mu**2 - lam:.6f}, μ ≈ {mu:.6f}, m1 ≈ {m1:.6f}, m2 ≈ {m2:.6f}, sub_min = {np.min(sub_phi)}, sup_max = {np.max(super_phi)} min φ ≈ {phi_min:.6f}")

    axs[idx].plot(x_vals, phi_vals_normalized, label="φ(x)")
    axs[idx].plot(x_vals, sub_phi, '--', label="Subsolution")
    axs[idx].plot(x_vals, super_phi, '--', label="Supersolution")
    axs[idx].set_title(f"λ = {lam}")
    axs[idx].axhline(0, color='gray', linestyle='--')
    axs[idx].legend()
    axs[idx].grid(True)

plt.tight_layout()
plt.show()

# Generate a fine range of lambda values
lambda_range = np.linspace(2.8, 50, 100)  # Adjust range as needed
super_phi_max_vals = []

for lam in lambda_range:
    mu = find_mu(lam, gamma1)
    if mu is None:
        super_phi_max_vals.append(np.nan)
        continue
    C = np.sqrt(lam) * gamma1 / mu
    phi = lambda x: np.cos(mu * x) + C * np.sin(mu * x)
    x_vals = np.linspace(0, 1, 1000)
    phi_vals = phi(x_vals)
    phi_vals_normalized = phi_vals / np.max(np.abs(phi_vals))
    phi_min = np.min(phi_vals_normalized)

    m2 = (-(mu**2 - lam) / (lam * phi_min))
    super_phi = phi_vals_normalized * m2
    super_phi_max_vals.append(np.max(super_phi))

# Plot super_phi max vs lambda
plt.figure(figsize=(8, 5))
plt.plot(lambda_range, super_phi_max_vals, label="max(super_phi)", color='blue')
plt.xlabel("λ")
plt.ylabel("max(super_phi)")
plt.title("Max of Supersolution vs λ")
plt.grid(True)
plt.legend()
plt.tight_layout()
plt.show()

In [ ]:
def eigenfunction_root(mu, lam, gamma1):
    term1 = (lam * gamma1**2 - mu**2) * np.sin(mu)
    term2 = 2 * np.sqrt(lam) * gamma1 * mu * np.cos(mu)
    return term1 + term2

def compute_subsolution(lam, gamma1, resolution=1000):
    # Find mu numerically
    mus = np.linspace(0.1, 20, 1000)
    for i in range(len(mus) - 1):
        a, b = mus[i], mus[i + 1]
        if eigenfunction_root(a, lam, gamma1) * eigenfunction_root(b, lam, gamma1) < 0:
            result = root_scalar(eigenfunction_root, args=(lam, gamma1), bracket=(a, b), method='brentq')
            if result.converged:
                mu = result.root
                break
    else:
        raise ValueError(f"Could not find a valid mu for λ={lam}")

    # Define normalized phi(x) = cos(mu x) + C sin(mu x)
    C = np.sqrt(lam) * gamma1 / mu
    x_vals = np.linspace(0, 1, resolution)
    phi_vals = np.cos(mu * x_vals) + C * np.sin(mu * x_vals)
    phi_vals /= np.max(np.abs(phi_vals))  # Normalize

    # Construct subsolution
    m1 = (-(mu**2 - lam) / lam) * 0.5
    m2 = -(mu**2 - lam) / (lam * np.min(phi_vals))
    sup_phi = phi_vals * m2
    sub_phi = phi_vals * m1
    sub_min = np.min(sub_phi)
    sup_max = np.max(sup_phi)

    return sub_min, sup_max

# (lambda, q, rho, sub_min, sup_max)
input_list = [
    (3.36251543, 0.12950433, 0.19183673),
    (5.84805224, 0.28840600, 0.46734694),
    (9.53230192, 0.36987505, 0.65102041),
    (19.37997763, 0.42421198, 0.83469388),
    (35.26398748, 0.43838751, 0.92653061)
]

output_list = []

for entry in input_list:
    lambd = entry[0]  # First element: lambda
    q = entry[1]      # Second element: q
    rho = entry[2]    # Third element: rho

    # Compute sub_min and sup_max for this lambda
    sub_min, sup_max = compute_subsolution(lambd, gamma1)
    print(f"sub_min ≈ {sub_min:.6f}, sup_max ≈ {sup_max:.6f}, lambda = {lambd}, q = {q}, ρ = {rho}")

    # Append full tuple (lambda, q, rho, sub_min, sup_max)
    output_list.append((lambd, q, rho, sub_min, sup_max))

# Define x values for plotting
x = np.linspace(-6, 6, 500)

save_indices = [0, 2, 4]  # indices of plots you want to save

for i, (lambd, q, rho, sub_min, sup_max) in enumerate(output_list):
    fig, ax = plt.subplots(figsize=(6, 4))  # One figure per plot

    sig = expit(x)
    scaled_sig = sub_min + (sup_max - sub_min) * sig

    # Plot scaled sigmoid
    ax.plot(x, scaled_sig, label='Scaled Sigmoid', color='blue')

    # Horizontal lines
    ax.axhline(q, color='red', linestyle='--', linewidth=1.5, label='q')
    ax.axhline(rho, color='green', linestyle='--', linewidth=1.5, label='rho')

    # Title and labels
    ax.set_title(f"γ = {gamma1} and λ = {lambd}")
    ax.set_xlabel("x")
    ax.set_ylabel("u(x)")
    ax.set_ylim(0, 3)
    ax.grid(True)
    ax.legend()

    plt.tight_layout()
    plt.show()

In [ ]:
# Step 1: Find super and sub solution values from eigenfunction
def eigenfunction_root(mu, lam, gamma1=1):
    term1 = (lam * gamma1**2 - mu**2) * np.sin(mu)
    term2 = 2 * np.sqrt(lam) * gamma1 * mu * np.cos(mu)
    return term1 + term2

def compute_subsolution(lam, gamma1=1, resolution=1000):
    # Find mu numerically
    mus = np.linspace(0.1, 20, 1000)
    for i in range(len(mus) - 1):
        a, b = mus[i], mus[i + 1]
        if eigenfunction_root(a, lam, gamma1=1) * eigenfunction_root(b, lam, gamma1) < 0:
            result = root_scalar(eigenfunction_root, args=(lam, gamma1), bracket=(a, b), method='brentq')
            if result.converged:
                mu = result.root
                break
    else:
        raise ValueError(f"Could not find a valid mu for λ={lam}")

    # Define normalized phi(x) = cos(mu x) + C sin(mu x)
    C = np.sqrt(lam) * gamma1 / mu
    x_vals = np.linspace(0, 1, resolution)
    phi_vals = np.cos(mu * x_vals) + C * np.sin(mu * x_vals)
    phi_vals /= np.max(np.abs(phi_vals))  # Normalize

    # Construct subsolution
    m1 = (-(mu**2 - lam) / lam) * 0.5
    m2 = -(mu**2 - lam) / (lam * np.min(phi_vals))
    sup_phi = phi_vals * m2
    sub_phi = phi_vals * m1
    sub_min = np.min(sub_phi)
    sub_max = np.max(sub_phi)
    sup_min = np.min(sup_phi)
    sup_max = np.max(sup_phi)

    return x_vals, sub_phi, sub_min, mu, sub_max, sup_phi, sup_max, sup_min

x_vals, sub_phi_vals, sub_min, mu, sub_max, sup_phi, sup_max, sup_min = compute_subsolution(lambd)
print(f"mu ≈ {mu:.6f}, sub_min ≈ {sub_min:.6f}, sub_max ≈ {sub_max:.6f}, sup_min ≈ {sup_min:.6f}, sup_max ≈ {sup_max:.6f}, lambda = {lambd}, q = {u0}, ρ = {rho}")

In [ ]:
class PeriodicActivation(nn.Module):
    def __init__(self, in_features, out_features, is_first=False):
        super().__init__()
        self.is_first = is_first
        self.linear = nn.Linear(in_features, out_features)

    def forward(self, x):
      x = self.linear(x)
      return 0.5 * (1 - torch.cos(2 * np.pi * x))

In [ ]:
class Net(nn.Module):
    def __init__(self, sub_min, sup_max):
        super(Net, self).__init__()
        self.sub_min = sub_min
        self.sup_max = sup_max

        self.hidden = nn.Sequential(
            PeriodicActivation(1, 40),
            PeriodicActivation(40, 60),
            PeriodicActivation(60, 20),
            nn.Linear(20, 1)  # Final raw output before scaling
        )
        self.output_activation = nn.Sigmoid()  # Maps to [0,1]

    def forward(self, t):
        raw = self.hidden(t)
        # Scale sigmoid to desired range
        return self.sub_min + (self.sup_max - self.sub_min) * self.output_activation(raw)

In [ ]:
def loss_function(u_pred, t, net):
    u_t = u_pred  # u(t)

    # First derivative u'(t)
    u_t_grad = torch.autograd.grad(u_pred, t, torch.ones_like(u_pred), create_graph=True)[0]

    # Second derivative u''(t)
    u_t_grad2 = torch.autograd.grad(u_t_grad, t, torch.ones_like(u_t_grad), create_graph=True, allow_unused=True)[0]

    # Define the differential equation residual: -u''(t) - lambda*u(t)(1 - u(t))
    f = -u_t_grad2 - lambd * u_t * (1 - u_t)

    # Compute the loss for the differential equation
    loss_eq = torch.mean(f**2)

    # Enforce Robin boundary conditions
    t_0 = torch.tensor([[0.0]], dtype=torch.float32, requires_grad=True)
    t_1 = torch.tensor([[1.0]], dtype=torch.float32, requires_grad=True)
    u_0 = net(t_0)
    u_1 = net(t_1)

    u_0_grad = torch.autograd.grad(u_0, t_0, torch.ones_like(u_0), create_graph=True)[0]
    u_1_grad = torch.autograd.grad(u_1, t_1, torch.ones_like(u_1), create_graph=True)[0]

    # Robin boundary conditions
    loss_bc_0 = ((-u_0_grad + torch.sqrt(torch.tensor(lambd)) * gamma1 * u_0) ** 2).mean()
    loss_bc_1 = ((u_1_grad + torch.sqrt(torch.tensor(lambd)) * gamma1 * u_1) ** 2).mean()

    return loss_eq, loss_bc_1, loss_bc_0

In [ ]:
def train_model(net, optimizer, scheduler=None, n_epochs=3000, print_every=500):
    t_values = torch.linspace(0, 1, 500).view(-1, 1)
    t_values.requires_grad_()

    for epoch in range(n_epochs):
        optimizer.zero_grad()
        u_pred = net(t_values)
        loss_eq, loss_bc_0, loss_bc_1 = loss_function(u_pred, t_values, net)

        total_loss = loss_eq + loss_bc_0 + loss_bc_1

        # Backpropagation and optimization step
        total_loss.backward()
        # Clipping gradients
        torch.nn.utils.clip_grad_norm_(net.parameters(), max_norm=1.0)
        optimizer.step()
        if scheduler is not None:
            scheduler.step()

        # Print the loss at regular intervals
        # if epoch % print_every == 0:
        #     print(f"Epoch {epoch}, Loss: {total_loss.item()}, EqLoss: {loss_eq.item()}, BC0Loss: {loss_bc_0.item()}, BC1Loss: {loss_bc_1.item()}")
    return net(t_values).detach().numpy().flatten()

In [ ]:
t_test = np.linspace(0, 1, 500)

net = Net(sub_min=sub_min, sup_max=sup_max).to(device)
optimizer = optim.Adam(net.parameters(), lr=0.001)
output = train_model(net, optimizer, n_epochs=11000)

def F(u):
    return 0.5 * u**2 - (1/3) * u**3

def ode1(t, u, rho, lambd):
    val = 2 * lambd * (F(rho) - F(u))
    return np.sqrt(np.maximum(val, 0))

sol1 = solve_ivp(ode1, [0, 0.5], [u0], args=(rho, lambd), dense_output=True, rtol=1e-9, atol=1e-12)
t1 = np.linspace(0, 0.5, 100)
u1 = sol1.sol(t1)
t2 = 1 - t1
u2 = u1[:, ::-1]

t_quad = np.concatenate([t1, t2[::-1]])
u_quad = np.concatenate([u1.flatten(), u2.flatten()])

In [ ]:
plt.figure(figsize=(10, 6))
plt.plot(t_test, output, label=f"NN Solution (λ={lambd})", color="red")
plt.plot(t_quad, u_quad, label="Quadrature Solution", color="blue", linewidth=2)
plt.title(f"Comparison: Neural Network vs Quadrature (λ={lambd})")
plt.xlabel("t")
plt.ylabel("u(t)")
plt.ylim(0, 1)
plt.grid(True)
plt.legend()
plt.tight_layout()
plt.show()